<a href="https://colab.research.google.com/github/yajima-yasutoshi/DataMinig/blob/main/20231108/%E3%82%AF%E3%83%A9%E3%82%B9%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データマイニング第7回（2023/11/8）

#本日の講義の目的

クラスタリング手法に関する説明を行う

## 本日の講義の資料

以下のサイトに保存してある「利用回数.xlsx」をダウンロード使う。

https://github.com/yajima-yasutoshi/DataMinig/tree/main/20231108



# 準備

In [ ]:
# インストール
!pip install japanize-matplotlib

## 必要なライブラリーのインポート

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

# クラスタリングとは


まず、以下のサイトより「利用回数.xlsx」をダウンロードし、Google Colab に読み込みを行う。

https://github.com/yajima-yasutoshi/DataMinig/tree/main/20231108

In [ ]:
# 読み込むファイルを指定する
file_path = '/content/drive/MyDrive/周南公立大学/講義/データマイニング/データ/利用回数.xlsx'
df = pd.read_excel(file_path)

In [ ]:
df.info()

サンプルとして用いるデータは、顧客毎に**年齢**とあるサービスの**利用回数**が記録されているものである。

In [ ]:
df.head()

年齢と利用回数の関係を散布図で可視化する。

In [ ]:
sns.scatterplot(x='年齢', y='利用回数', data=df)
plt.title('散布図')
plt.show()

上の散布図から、
 * 年齢が高く利用回数の多い利用者
 * 年齢が低く利用回数の多い利用者
 * 年齢が低く利用回数の少ない利用者

に傾向が分かれていることが観察できる。

傾向が似通っているレコードを自動的に分類する方法が
**「クラスタリング」**である。

スーパーやドラッグストア、コンビニなどの小売店などでは、
顧客の属性や購買行動のデータを元にして、
傾向が似ている顧客をいくつかのグループに分類し、
どのような顧客が良く利用しているのか、
どのような商品を購入する顧客がいるのか、
など、顧客の行動を理解することで販促や商品開発などを行っている。

参考事例：
https://www.nikkei.com/article/DGXZQOUC132X30T10C22A9000000/

## クラスタリングの実行手順

### データを標準化する。
 クラスタリングを実行する前にデータを**標準化**し、
 データの大きさを平均 0、分散 1 に変換する必要がある。

In [ ]:
fig, (ax1, ax2) = plt.subplots( 1, 2 , figsize=(10,4))
sns.histplot( df['年齢'], bins=10, kde=False, ax=ax1)
sns.histplot( df['利用回数'], bins=10, kde=False, ax=ax2)
plt.show()

In [ ]:
df['年齢'].var()

In [ ]:
df['利用回数'].var()

このように、項目によって数値の大きさが異なる場合には標準化が不可欠である。

var()以外にもデータ分析で使う主な関数には以下のものがある。

関数  | 意味
--    |   --
sum()    | 合計
mean()   | 平均
**var()**    | **分散**
std()    | 標準偏差
median() | 中央値
min()    | 最小値
max()    | 最大値
quantile(0.25)  | 四分位点

### 分散と標準偏差

標準偏差を二乗すると分散となる。例えば、以下のように確認できる。

In [ ]:
df['利用回数'].std() * df['利用回数'].std()

## Pythonでの前処理

In [ ]:
# 標準化に必要なライブラリーのインポート
from sklearn.preprocessing import StandardScaler

In [ ]:
# 標準化の対象となる項目を変数 X にセットする
X = df[['年齢', '利用回数']]

# 準備の準備
scaler = StandardScaler()

# 標準化の実施
# 標準化後のデータを再びXにセットする
X = scaler.fit_transform(X)

In [ ]:
# 標準化の確認
X = pd.DataFrame( X, columns=['x1', 'x2'])

fig, (ax1, ax2) = plt.subplots( 1, 2 , figsize=(10,4))

ax1.set_title("変換後")
#sns.histplot(X[:,0], ax=ax1)
sns.scatterplot(data=X, x='x1', y='x2', ax=ax1)

ax2.set_title("変換前")
#sns.histplot(df['年齢'], ax=ax2)
sns.scatterplot(data=df, x='年齢', y='利用回数', ax=ax2 )

### 標準化の確認

In [ ]:
# 元のデータの平均を計算
df['年齢'].mean()

In [ ]:
# 元のデータの分散を計算
df['年齢'].var()

In [ ]:
# 標準化後のデータの平均
X['x1'].mean()

In [ ]:
# 標準化後のデータの分散を計算
X['x1'].var()

## クラスタリングの実行

本講義では、クラスタリングを行う代表的な手法であるK-means法を扱う。

データ分析ライブラリーの sklearn に含まれている**KMeans**を用いる。
詳しい使い方は以下のリンクを参照

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html


In [ ]:
# クラスタリングに必要なライブラリーのインポート
from sklearn.cluster import KMeans

# 標準化に必要なライブラリーのインポート
from sklearn.preprocessing import StandardScaler

# data という変数に読み込む
df = pd.read_excel(file_path)

# クラスタリング分析に用いる項目を選ぶ
col = ['年齢', '利用回数']

# 選んだ項目を変数 X にセットする
X = df[col]

# 準備の準備
scaler = StandardScaler()

# 標準化後のデータを再びXにセットする
X = scaler.fit_transform(X)

# 最適なクラスタ数を3であると仮定
best_n_clusters = 3

kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', max_iter=500, n_init='auto')
kmeans.fit(X)

 kmeans.labels_ にクラスタリングの結果が格納されているので、それをdfに追加する。

In [ ]:
# クラスタラベルをデータフレームに追加
df['cluster'] = kmeans.labels_

In [ ]:
df.head()

In [ ]:
# 結果の可視化（seabornを使用）
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
plt.show()

In [ ]:
# クラスタで分類して年齢と利用回数の平均を求める
# クラスタ中心と呼ぶ
df.groupby('cluster')[['年齢','利用回数']].mean()

In [ ]:
c = df.groupby('cluster')[['年齢','利用回数']].mean().reset_index()
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
sns.scatterplot(x='年齢', y='利用回数', data=c, color='k', s=100)

In [ ]:
# クラスタ毎にクラスタ中心から各データへの距離の二乗の和
kmeans.inertia_

## クラスタ数を変えて可視化する

In [ ]:
# クラスタ数を 2 にして実行する
best_n_clusters = 2

kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', max_iter=500, n_init='auto')
kmeans.fit(X)
df['cluster'] = kmeans.labels_

# 図示する
c = df.groupby('cluster')[['年齢','利用回数']].mean().reset_index()
sns.scatterplot(x='年齢', y='利用回数', hue='cluster', data=df, palette='Set1')
sns.scatterplot(x='年齢', y='利用回数', data=c, color='k', s=100)

In [ ]:
# クラスタ毎にクラスタ中心から各データへの距離の二乗の和
kmeans.inertia_

## エルボー法

クラスタ数を決める方法として、エルボー法が使われる。

クラスタ数を変化させながら、クラスタ中心からの距離の二乗の和を計算しグラフ化する。
グラフの減少度合いが変化し始めるところを、最適なクラスター数として採用する。

こういった作業はAIの開発では一般的で、**「ハイパーパラメータチューニング」**と呼ばれる。

In [ ]:
# Elbow Methodによるハイパーパラメータチューニング
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init='auto')
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
# グリッド線を表示するスタイルに変更する
sns.set_style('whitegrid')

# エルボー図のプロット
elbow = pd.DataFrame( inertia, columns = ['Inertia'])
elbow['Num of clusters'] = range(1,11)
sns.lineplot(data = elbow, x='Num of clusters', y= 'Inertia')

# Wineデータをつかった例

In [ ]:
from sklearn.datasets import load_wine
# データの読み込み
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

In [ ]:
df.info()

## 利用するデータ概要

Wineデータセットは、イタリアの同じ地域で栽培された3種類のワインに
含まれる成分を測定したデータ。
13の項目がある。

| 項目                          | 説明                                       |
|---------------------------------|--------------------------------------------|
| Alcohol（アルコール）           | ワインに含まれるアルコールの量。           |
| Malic Acid（リンゴ酸）          | ワインに含まれるリンゴ酸の量。             |
| Ash（灰分）                     | ワインの灰分の量。                         |
| Alcalinity of Ash（灰分のアルカリ度） | 灰分のアルカリ度。                   |
| Magnesium（マグネシウム）       | ワインに含まれるマグネシウムの量。         |
| Total Phenols（総フェノール）   | ワインに含まれるフェノール類の総量。       |
| Flavanoids（フラバノイド）      | ワインに含まれるフラバノイドの量。         |
| Nonflavanoid Phenols（非フラバノイドフェノール） | ワインに含まれる非フラバノイドフェノールの量。 |
| Proanthocyanins（プロアントシアニン） | ワインに含まれるプロアントシアニンの量。 |
| Color Intensity（色の強度）     | ワインの色の強度。                         |
| Hue（色相）                     | ワインの色相。                             |
| OD280/OD315 of Diluted Wines（希釈ワインのOD280/OD315） | 希釈されたワインのOD280/OD315の比率。 |
| Proline（プロリン）             | ワインに含まれるプロリンの量。             |

In [ ]:
# 相関行列を計算
correlation_matrix = df.corr().abs()
# sns.heatmap(correlation_matrix, cmap= sns.color_palette('coolwarm', 10), annot=True,fmt='.2f', vmin = -1, vmax = 1)

# 相関が0.5以上の変数を取得
high_corr_var = {}
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= 0.5:
            colname = correlation_matrix.columns[i]
            high_corr_var[colname] = abs(correlation_matrix.iloc[i, j])

# 相関性の高い変数を削除
df.drop(columns=high_corr_var.keys(), inplace=True)

# 結果の表示
print("Remaining columns after removing highly correlated variables:")
print(df.columns)
df.info()

簡単にするため以下の項目限定して分析する。

| 項目                          | 説明                                       |
|---------------------------------|--------------------------------------------|
| Alcohol（アルコール）           | ワインに含まれるアルコールの量。           |
| Malic Acid（リンゴ酸）          | ワインに含まれるリンゴ酸の量。             |
| Ash（灰分）                     | ワインの灰分の量。                         |
| Alcalinity of Ash（灰分のアルカリ度） | 灰分のアルカリ度。                   |
| Magnesium（マグネシウム）       | ワインに含まれるマグネシウムの量。         |
| Total Phenols（総フェノール）   | ワインに含まれるフェノール類の総量。       |


In [ ]:
df = df[['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium','total_phenols']]
df.info()

In [ ]:
sns.pairplot( data = df)

In [ ]:
# 標準化に必要なライブラリーのインポート
from sklearn.preprocessing import StandardScaler
# 前処理（データの正規化）
X = df
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# クラスタリングに必要なライブラリーのインポート
from sklearn.cluster import KMeans


# Elbow Methodによるハイパーパラメータチューニング
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=500, n_init='auto')
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
# グリッド線を表示する場合は以下を行う
sns.set_style('whitegrid')

# エルボー図のプロット
elbow = pd.DataFrame( inertia, columns = ['Inertia'])
elbow['Num of clusters'] = range(1,11)
sns.lineplot(data = elbow, x='Num of clusters', y= 'Inertia')

In [ ]:
# 最適なクラスタ数でKMeansクラスタリング（この例では、エルボーが3であると仮定）
best_n_clusters = 3  # エルボー図から選択
kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', max_iter=300, n_init='auto')
kmeans.fit(X)

# クラスタラベルをデータフレームに追加
df['cluster'] = kmeans.labels_

# 各クラスタの特徴を調査（平均値、標準偏差など）
cluster_summary = df.groupby('cluster').agg(['mean']).reset_index()

# 結果の表示
cluster_summary


In [ ]:
# 結果の可視化（seabornを使用）
sns.scatterplot(x='alcohol', y='total_phenols', hue='cluster', data=df, palette='Set1')
plt.title('Clustering Results with Wine Data')
plt.show()

In [ ]:
sns.pairplot( data = df, hue='cluster', palette='Set1')